In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
import time
from yahooquery import Ticker

In [ ]:
import pandas as pd
import requests
from yahooquery import Ticker

# ==========================================
# 1. FETCH UNIVERSE (Raw List) for Canadian Markets
# ==========================================
def get_canadian_universe_robust():
    print("--- STEP 1: Fetching TSX & TSX-V Stock List ---")
    tickers = []
    
    # 1. Try Fetching from TMX
    url = "https://www.tsx.com/files/trading/moc-eligible-stocks.txt"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            lines = response.text.split('\n')
            for line in lines:
                parts = line.split()
                if len(parts) < 2: continue
                
                exchange = parts[0]
                symbol = parts[1]
                
                # Yahoo Format Conversion
                if exchange == 'TSX':
                    tickers.append(f"{symbol.replace('.', '-')}.TO")
                elif exchange == 'TSXV':
                    tickers.append(f"{symbol.replace('.', '-')}.V")
                    
        print(f"   -> Scraper found {len(tickers)} candidates.")
    except Exception as e:
        print(f"   -> Web scrape failed ({e}).")

    # 2. Backup List
    if len(tickers) == 0:
        print("   -> Using Backup List...")
        tickers = [
            'SHOP.TO', 'CSU.TO', 'ATD.TO', 'DOL.TO', 'L.TO', 'WN.TO', 'EMP-A.TO', 'MRU.TO',
            'CNR.TO', 'CP.TO', 'TFII.TO', 'WCN.TO', 'CAE.TO', 'AC.TO',
            'SU.TO', 'CNQ.TO', 'CVE.TO', 'IMO.TO', 'TOU.TO', 'ARX.TO',
            'NTR.TO', 'TECK-B.TO', 'FM.TO', 'CCO.TO', 'WPM.TO', 'AEM.TO', 'ABX.TO',
            'OTEX.TO', 'GIB-A.TO', 'KXS.TO', 'DSG.TO',
            'BCE.TO', 'T.TO', 'RCI-B.TO', 'QBR-B.TO',
            'FTS.TO', 'EMA.TO', 'H.TO', 'AQN.TO', 'NPI.TO'
        ]

    return list(set(tickers))

# ==========================================
# 2. BUFFETT SCAN (With Price/Vol Filters)
# ==========================================
def run_buffett_nav_scan_filtered(ticker_list):
    """
    Scans for companies trading BELOW Book Value (P/B < 1) 
    that are still profitable (ROE > 0) AND meet liquidity requirements.
    """
    print(f"\n--- STEP 2: Running 'Buffett NAV' Filter ---")
    print("   Criteria: Price > $3 | Cap > $50M | Vol > 50k")
    print("   Value Criteria: P/B < 1.0 | ROE > 0% | Debt/Eq < 100%")
    
    buffett_picks = []
    chunk_size = 300 
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Scanning batch {i} - {min(i+chunk_size, len(ticker_list))}...", end='\r')
        
        try:
            # Asynchronous fetch
            yq = Ticker(chunk, asynchronous=True)
            data = yq.get_modules("defaultKeyStatistics financialData price summaryProfile summaryDetail")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): continue
                
                # --- DATA EXTRACTION ---
                price_mod = data[symbol].get('price', {})
                summ_det = data[symbol].get('summaryDetail', {})
                stats = data[symbol].get('defaultKeyStatistics', {})
                fin = data[symbol].get('financialData', {})
                profile = data[symbol].get('summaryProfile', {})
                
                # --- 1. PRE-FILTERS (Liquidity & Size) ---
                price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                avg_vol = summ_det.get('averageVolume', 0) or 0
                
                # FILTER: Price > $3
                if price < 3.0: continue
                
                # FILTER: Market Cap > $50 Million
                if mkt_cap < 50_000_000: continue
                
                # FILTER: Avg Volume > 50,000
                if avg_vol < 50_000: continue

                # --- 2. VALUE CRITERIA (Buffett Logic) ---
                
                # P/B < 1.0 (Trading under NAV)
                pb_ratio = stats.get('priceToBook')
                if pb_ratio is None: continue
                if pb_ratio >= 1.0: continue 
                if pb_ratio <= 0: continue # Skip insolvent
                
                # ROE > 0 (Must be Profitable)
                roe = fin.get('returnOnEquity', 0) or 0
                if roe <= 0: continue
                
                # Debt/Equity < 100% (Safety)
                debt_equity = fin.get('debtToEquity', 0) or 0
                if debt_equity > 100: continue 

                # --- CAPTURE ---
                sector = profile.get('sector', 'Unknown')
                
                buffett_picks.append({
                    'Ticker': symbol,
                    'Price': price,
                    'P/B Ratio': round(pb_ratio, 2),
                    'ROE %': round(roe * 100, 2),
                    'Debt/Eq %': round(debt_equity, 2),
                    'Market Cap (M)': round(mkt_cap / 1_000_000, 1),
                    'Vol': avg_vol,
                    'Sector': sector
                })
                        
        except Exception as e:
            continue
            
    df = pd.DataFrame(buffett_picks)
    
    if not df.empty:
        print(f"\n   -> Found {len(df)} Buffett-style value plays.")
        # Sort by P/B Ratio (Deepest Value First)
        return df.sort_values(by='P/B Ratio', ascending=True)
    else:
        print(f"\n   -> No stocks found matching criteria.")
        return pd.DataFrame()

# ==========================================
# MAIN EXECUTION
# ==========================================

# 1. Get Raw Universe
raw_tickers = get_canadian_universe_robust()

# 2. Run Filtered Buffett Scan
buffett_results = run_buffett_nav_scan_filtered(raw_tickers)

# 3. Display Results
if not buffett_results.empty:
    print("\n\n" + "="*60)
    print("WARREN BUFFETT SCREEN (Price > $3, Cap > $50M, Vol > 50k)")
    print("="*60)
    
    # Display columns
    cols = ['Ticker', 'Price', 'P/B Ratio', 'ROE %', 'Debt/Eq %', 'Market Cap (M)', 'Vol', 'Sector']
    
    # Show top 25 results
    try:
        display(buffett_results[cols].head(25))
    except:
        print(buffett_results[cols].head(25))
else:
    print("No results found.")

--- STEP 1: Fetching TSX & TSX-V Stock List ---
   -> Scraper found 0 candidates.
   -> Using Backup List...

--- STEP 2: Running 'Buffett NAV' Filter ---
   Criteria: Price > $3 | Cap > $50M | Vol > 50k
   Value Criteria: P/B < 1.0 | ROE > 0% | Debt/Eq < 100%
   Scanning batch 0 - 40...
   -> No stocks found matching criteria.
No results found.


In [ ]:
import pandas as pd
import requests
from yahooquery import Ticker
import time

# ==========================================
# 1. FETCH US UNIVERSE (S&P 500 + NASDAQ 100)
# ==========================================
def get_us_universe_robust():
    print("--- STEP 1: Fetching US Market Tickers (S&P 500 + NASDAQ 100) ---")
    tickers = []
    
    # Define a "Fake Browser" Header to bypass Wikipedia 403 blocks
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        # 1. Fetch S&P 500 from Wikipedia (Using Requests + Headers)
        url_sp500 = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
        response_sp = requests.get(url_sp500, headers=headers)
        
        # Parse the text response with Pandas
        tables_sp = pd.read_html(response_sp.text)
        sp500_df = tables_sp[0]
        sp500_tickers = sp500_df['Symbol'].tolist()
        tickers.extend(sp500_tickers)
        print(f"   -> Fetched {len(sp500_tickers)} S&P 500 stocks.")
        
        # 2. Fetch NASDAQ 100 from Wikipedia (Using Requests + Headers)
        url_ndx = "https://en.wikipedia.org/wiki/Nasdaq-100"
        response_ndx = requests.get(url_ndx, headers=headers)
        
        tables_ndx = pd.read_html(response_ndx.text)
        
        # Loop to find the correct table (look for 'Ticker' or 'Symbol')
        for t in tables_ndx:
            if 'Ticker' in t.columns:
                ndx_tickers = t['Ticker'].tolist()
                tickers.extend(ndx_tickers)
                break
            elif 'Symbol' in t.columns:
                ndx_tickers = t['Symbol'].tolist()
                tickers.extend(ndx_tickers)
                break
                
    except Exception as e:
        print(f"   -> Web scrape failed ({e}). Using backup list.")

    # 3. Clean Tickers
    # Yahoo Finance expects 'BRK-B', not 'BRK.B'
    clean_tickers = [str(t).replace('.', '-') for t in tickers]
    
    # Remove duplicates
    final_list = list(set(clean_tickers))
    
    # 4. BACKUP LIST (Safety Net)
    if len(final_list) == 0:
        print("   -> Using Backup List of Top US Stocks...")
        final_list = [
            'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'TSLA', 'META', 'BRK-B',
            'JPM', 'JNJ', 'V', 'PG', 'MA', 'HD', 'CVX', 'MRK', 'ABBV',
            'KO', 'PEP', 'BAC', 'COST', 'WMT', 'MCD', 'DIS', 'CSCO',
            'VZ', 'CMCSA', 'ADBE', 'NKE', 'INTC', 'T', 'PFE', 'WFC',
            'C', 'F', 'GM', 'XOM', 'CVX', 'O', 'MMM', 'IBM', 'GE'
        ]

    print(f"   -> Total Universe Size: {len(final_list)} unique stocks.")
    return final_list

# ==========================================
# 2. BUFFETT SCAN (US Version)
# ==========================================
def run_buffett_us_scan(ticker_list):
    """
    Scans for US companies trading BELOW Book Value (P/B < 1) 
    that are profitable (ROE > 0) and Liquid.
    """
    print(f"\n--- STEP 2: Running 'Buffett NAV' Filter (US Market) ---")
    print("   Criteria: Price > $3 | Cap > $50M | Vol > 50k")
    print("   Value Criteria: P/B < 1.0 | ROE > 0% | Debt/Eq < 100%")
    
    buffett_picks = []
    chunk_size = 300 
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Scanning batch {i} - {min(i+chunk_size, len(ticker_list))}...", end='\r')
        
        try:
            # Asynchronous fetch
            yq = Ticker(chunk, asynchronous=True)
            data = yq.get_modules("defaultKeyStatistics financialData price summaryProfile summaryDetail")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): continue
                
                # --- DATA EXTRACTION ---
                price_mod = data[symbol].get('price', {})
                summ_det = data[symbol].get('summaryDetail', {})
                stats = data[symbol].get('defaultKeyStatistics', {})
                fin = data[symbol].get('financialData', {})
                profile = data[symbol].get('summaryProfile', {})
                
                # --- 1. PRE-FILTERS (Liquidity & Size) ---
                price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                avg_vol = summ_det.get('averageVolume', 0) or 0
                
                # FILTER: Price > $3
                if price < 3.0: continue
                
                # FILTER: Market Cap > $50 Million USD
                if mkt_cap < 50_000_000: continue
                
                # FILTER: Avg Volume > 500,000
                if avg_vol < 500_000: continue

                # --- 2. VALUE CRITERIA (Buffett Logic) ---
                
                # P/B < 1.0 (Trading under NAV)
                pb_ratio = stats.get('priceToBook')
                if pb_ratio is None: continue
                if pb_ratio >= 1.0: continue 
                if pb_ratio <= 0: continue # Skip insolvent
                
                # ROE > 0 (Must be Profitable)
                roe = fin.get('returnOnEquity', 0) or 0
                if roe <= 0: continue
                
                # Debt/Equity < 100% (Safety)
                debt_equity = fin.get('debtToEquity', 0) or 0
                if debt_equity > 100: continue 

                # --- CAPTURE ---
                sector = profile.get('sector', 'Unknown')
                
                buffett_picks.append({
                    'Ticker': symbol,
                    'Price': price,
                    'P/B Ratio': round(pb_ratio, 2),
                    'ROE %': round(roe * 100, 2),
                    'Debt/Eq %': round(debt_equity, 2),
                    'Market Cap (B)': round(mkt_cap / 1_000_000_000, 2), # Billions for US
                    'Sector': sector
                })
                        
        except Exception as e:
            continue
            
    df = pd.DataFrame(buffett_picks)
    
    if not df.empty:
        print(f"\n   -> Found {len(df)} Buffett-style value plays.")
        # Sort by P/B Ratio (Deepest Value First)
        return df.sort_values(by='P/B Ratio', ascending=True)
    else:
        print(f"\n   -> No stocks found matching criteria.")
        return pd.DataFrame()

# ==========================================
# MAIN EXECUTION
# ==========================================

# 1. Get US Universe
us_tickers = get_us_universe_robust()

# 2. Run Filtered Buffett Scan
us_buffett_results = run_buffett_us_scan(us_tickers)

# 3. Display Results
if not us_buffett_results.empty:
    print("\n\n" + "="*60)
    print("US MARKET: WARREN BUFFETT SCREEN (Price < Book)")
    print("="*60)
    
    # Display columns
    cols = ['Ticker', 'Price', 'P/B Ratio', 'ROE %', 'Debt/Eq %', 'Market Cap (B)', 'Sector']
    
    # Show top 25 results
    try:
        display(us_buffett_results[cols].head(25))
    except:
        print(us_buffett_results[cols].head(25))
else:
    print("No results found. (Note: US Large Caps rarely trade below Book Value)")

--- STEP 1: Fetching US Market Tickers (S&P 500 + NASDAQ 100) ---


C:\Users\James\AppData\Local\Temp\ipykernel_164\316647281.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_sp = pd.read_html(response_sp.text)


   -> Fetched 501 S&P 500 stocks.
   -> Total Universe Size: 516 unique stocks.

--- STEP 2: Running 'Buffett NAV' Filter (US Market) ---
   Criteria: Price > $3 | Cap > $50M | Vol > 50k
   Value Criteria: P/B < 1.0 | ROE > 0% | Debt/Eq < 100%


C:\Users\James\AppData\Local\Temp\ipykernel_164\316647281.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_ndx = pd.read_html(response_ndx.text)


   Scanning batch 300 - 516...
   -> Found 6 Buffett-style value plays.


US MARKET: WARREN BUFFETT SCREEN (Price < Book)


,Ticker,Price,P/B Ratio,ROE %,Debt/Eq %,Market Cap (B),Sector
2,BRK-B,498.30,0.00,10.17,18.17,1075.00,Financial Services
4,MOS,24.31,0.60,10.12,37.13,7.72,Basic Materials
3,DD,41.26,0.75,3.22,39.67,17.29,Basic Materials
5,GPN,80.35,0.84,7.23,69.20,19.49,Industrials
0,MSTR,158.81,0.87,25.59,14.15,45.63,Technology
1,EG,337.53,0.92,3.60,23.34,14.17,Financial Services


In [2]:
import pandas as pd
import requests
import io
import time
from yahooquery import Ticker

# ==============================================================================
# 1. FETCH FULL US UNIVERSE (NYSE, NASDAQ, AMEX) - ROBUST VERSION
# ==============================================================================
def get_us_universe_robust():
    """
    Fetches the complete list of US-traded stocks from NasdaqTrader.com.
    Includes 'Fake Browser' headers to bypass 403 Forbidden errors.
    Falls back to S&P 500 (Wikipedia) or a Manual List if the main source fails.
    """
    print("--- STEP 1: Fetching FULL US Market Tickers (NYSE, NASDAQ, AMEX) ---")
    tickers = []
    
    # --- CRITICAL: Fake Browser Header to bypass blocks ---
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        # 1. Fetch Traded List from NASDAQ FTP (Includes NYSE/AMEX)
        url = "http://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
        
        # Use requests with headers
        response = requests.get(url, headers=headers)
        response.raise_for_status() # Check for 403/404 errors
        
        # Parse CSV from string content
        df = pd.read_csv(io.StringIO(response.content.decode('utf-8')), sep='|')
        
        # 2. Filter Process
        # 'Test Issue' == 'N' removes test stocks
        # 'ETF' == 'N' removes ETFs (we want operating companies)
        df_clean = df[(df['Test Issue'] == 'N') & (df['ETF'] == 'N')]
        
        # 3. Symbol Cleanup (Yahoo uses '-' instead of '.')
        raw_list = df_clean['Symbol'].dropna().unique().tolist()
        clean_list = [str(x).replace('.', '-') for x in raw_list]
        
        # Filter: Length <= 5 (removes most warrants/rights) and no '$' signs
        final_list = [t for t in clean_list if len(t) <= 5 and '$' not in t]
        
        tickers.extend(final_list)
        print(f"   -> Fetched {len(tickers)} active US stocks (Source: NasdaqTrader).")
        
    except Exception as e:
        print(f"   -> Nasdaq source failed ({e}). Switching to S&P 500 backup...")
        
        # FALLBACK: Fetch S&P 500 from Wikipedia
        try:
            url_sp = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
            # We MUST use headers here too, or Wikipedia will block us
            response = requests.get(url_sp, headers=headers)
            
            tables = pd.read_html(response.text)
            tickers = tables[0]['Symbol'].str.replace('.', '-', regex=False).tolist()
            print(f"   -> Backup: Fetched {len(tickers)} S&P 500 stocks.")
        except Exception as e2:
            print(f"   -> Backup failed ({e2}). Using small manual list.")
            tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'BRK-B', 'JPM', 'NVDA', 'TSLA']

    return list(set(tickers))

# ==============================================================================
# 2. BUFFETT SCAN (US Version)
# ==============================================================================
def run_buffett_us_scan(ticker_list):
    """
    Scans for US companies trading BELOW Book Value (P/B < 1) 
    that are profitable (ROE > 0) and Liquid.
    """
    print(f"\n--- STEP 2: Running 'Buffett NAV' Filter (US Market) ---")
    print("   Criteria: Price > $3 | Cap > $50M | Vol > 500k")
    print("   Value Criteria: P/B < 1.0 | ROE > 0% | Debt/Eq < 100%")
    
    buffett_picks = []
    chunk_size = 500 # Increased chunk size for speed
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Scanning batch {i} - {min(i+chunk_size, len(ticker_list))}...", end='\r')
        
        try:
            # Asynchronous fetch
            yq = Ticker(chunk, asynchronous=True)
            data = yq.get_modules("defaultKeyStatistics financialData price summaryProfile summaryDetail")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): continue
                
                # --- DATA EXTRACTION ---
                price_mod = data[symbol].get('price', {})
                summ_det = data[symbol].get('summaryDetail', {})
                stats = data[symbol].get('defaultKeyStatistics', {})
                fin = data[symbol].get('financialData', {})
                profile = data[symbol].get('summaryProfile', {})
                
                # --- 1. PRE-FILTERS (Liquidity & Size) ---
                price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                avg_vol = summ_det.get('averageVolume', 0) or 0
                
                # FILTER: Price > $3 (Avoid penny stocks)
                if price < 3.0: continue
                
                # FILTER: Market Cap > $50 Million USD
                if mkt_cap < 50_000_000: continue

                # FILTER: Avg Volume > 500,000 (Ensure Liquidity)
                if avg_vol < 500_000: continue

                # --- 2. VALUE CRITERIA (Buffett Logic) ---
                
                # P/B < 1.0 (Trading under NAV)
                pb_ratio = stats.get('priceToBook')
                if pb_ratio is None: continue
                if pb_ratio >= 1.0: continue 
                if pb_ratio <= 0: continue # Skip insolvent
                
                # ROE > 0 (Must be Profitable)
                roe = fin.get('returnOnEquity', 0) or 0
                if roe <= 0: continue
                
                # Debt/Equity < 100% (Safety)
                debt_equity = fin.get('debtToEquity', 0) or 0
                if debt_equity > 100: continue 

                # --- CAPTURE ---
                sector = profile.get('sector', 'Unknown')
                
                buffett_picks.append({
                    'Ticker': symbol,
                    'Price': price,
                    'P/B Ratio': round(pb_ratio, 2),
                    'ROE %': round(roe * 100, 2),
                    'Debt/Eq %': round(debt_equity, 2),
                    'Market Cap (M)': round(mkt_cap / 1_000_000, 2), # Millions for broader view
                    'Sector': sector
                })
                        
        except Exception as e:
            continue
            
    df = pd.DataFrame(buffett_picks)
    
    if not df.empty:
        print(f"\n   -> Found {len(df)} Buffett-style value plays.")
        # Sort by P/B Ratio (Deepest Value First)
        return df.sort_values(by='P/B Ratio', ascending=True)
    else:
        print(f"\n   -> No stocks found matching criteria.")
        return pd.DataFrame()

# ==============================================================================
# 3. MAIN EXECUTION BLOCK
# ==============================================================================
if __name__ == "__main__":
    # 1. Get Full US Universe
    us_tickers = get_us_universe_robust()

    # 2. Run Filtered Buffett Scan
    us_buffett_results = run_buffett_us_scan(us_tickers)

    # 3. Display Results
    if not us_buffett_results.empty:
        print("\n\n" + "="*60)
        print("US MARKET (ALL): WARREN BUFFETT SCREEN (Price < Book)")
        print("="*60)
        
        # Display columns
        cols = ['Ticker', 'Price', 'P/B Ratio', 'ROE %', 'Debt/Eq %', 'Market Cap (M)', 'Sector']
        
        # Check if we are in a Notebook environment for pretty display, else print
        try:
            from IPython.display import display
            display(us_buffett_results[cols].head(10))
        except ImportError:
            print(us_buffett_results[cols].head(10))
            
        # Optional: Save to Excel
        # us_buffett_results.to_excel("Buffett_US_Picks.xlsx", index=False)
        # print("\nSaved to Buffett_US_Picks.xlsx")
    else:
        print("No results found.")

--- STEP 1: Fetching FULL US Market Tickers (NYSE, NASDAQ, AMEX) ---
   -> Fetched 6825 active US stocks (Source: NasdaqTrader).

--- STEP 2: Running 'Buffett NAV' Filter (US Market) ---
   Criteria: Price > $3 | Cap > $50M | Vol > 500k
   Value Criteria: P/B < 1.0 | ROE > 0% | Debt/Eq < 100%
   Scanning batch 6500 - 6825...
   -> Found 114 Buffett-style value plays.


US MARKET (ALL): WARREN BUFFETT SCREEN (Price < Book)


,Ticker,Price,P/B Ratio,ROE %,Debt/Eq %,Market Cap (M),Sector
35,BRK-B,500.078,0.00,10.17,18.17,1078835.68,Financial Services
20,KSPI,79.125,0.01,59.55,17.72,15920.52,Technology
101,TKC,5.590,0.05,6.65,74.59,4871.21,Communication Services
108,MFG,7.320,0.26,9.28,0.00,90753.90,Financial Services
64,IMPP,3.635,0.27,8.90,0.00,131.68,Energy
28,ERIC,9.690,0.31,26.64,42.87,32299.55,Technology
12,LX,3.295,0.33,16.41,40.32,554.43,Financial Services
59,CIVI,27.080,0.35,9.55,76.87,2310.29,Energy
1,GMAB,32.805,0.35,29.41,2.47,20336.95,Healthcare
7,HPK,4.705,0.36,3.29,73.38,593.45,Energy
